# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
#импортируем нужные нам библиотеки и алгоритмы
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.dummy import DummyClassifier

In [2]:
#считываем датасет
df = pd.read_csv('/datasets/users_behavior.csv')

#выведем первые 10 строк датафрейма
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [3]:
#просмотрим общую информацию о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Всего 3214 строк и 5 столбцов. Названия столбцов корректны. Пропусков нет. Типы данных соответствуют. Дубликаты искать не будем, т.к. все значения числовые. Предобработка не требуется.

## Разбейте данные на выборки

Данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. Исходные данные разобьем в соотношении 3:1:1. 

In [4]:
#сначала разобьем исходные данные на 2 группы, отделив обучающие данные от тестовых + валидационных
df_train, df_test_valid = train_test_split(df, test_size=0.40, random_state=12345)

#затем разобьем вторую группу пополам: одни данные будут валидационными, другие - тестовыми 
df_valid, df_test = train_test_split(df_test_valid, test_size=0.50, random_state=12345)

#проверим
display(df_train.shape)
display(df_valid.shape)
display(df_test.shape)

(1928, 5)

(643, 5)

(643, 5)

## Исследуйте модели

In [5]:
#выделим признаки и целевой признак из обучающей выборки
train_features = df_train.drop('is_ultra', 1)
train_target = df_train['is_ultra']

#выделим признаки и целевой признак из валидационной выборки
valid_features = df_valid.drop('is_ultra', 1)
valid_target = df_valid['is_ultra']

**Дерево решений**

Начнем с алгоритма "дерево решений"

In [6]:
#создаем объект
model_dtc = DecisionTreeClassifier(random_state=12345)

#обучаем модель
model_dtc.fit(train_features, train_target)

#получаем предсказание
valid_predictions_dtc = model_dtc.predict(valid_features)

#считаем метрику
valid_accuracy_dtc = accuracy_score(valid_target, valid_predictions_dtc)
valid_accuracy_dtc

0.713841368584759

Попробуем улучшить. Найдем оптимальную глубину дерева, например, из 100, которая дает лучшее значение `accuracy`.

In [7]:
#счетчик на лучшую глубину
best_max_depth = 0

#счетчик на лучшую accuracy
best_max_depth_accuracy = 0

#переберем каждое значение глубины от 1 до 100
for i in range(1,101):
    model_dtc1 = DecisionTreeClassifier(random_state=12345, max_depth=i)
    model_dtc1.fit(train_features, train_target)
    valid_predictions_dtc1 = model_dtc1.predict(valid_features)
    valid_accuracy_dtc1 = accuracy_score(valid_target, valid_predictions_dtc1)
    if valid_accuracy_dtc1 > best_max_depth_accuracy:
        best_max_depth = i
        best_max_depth_accuracy = valid_accuracy_dtc1

print('Лучшая accuracy', best_max_depth_accuracy, 'при глубине', best_max_depth)

Лучшая accuracy 0.7853810264385692 при глубине 3


In [8]:
model_dtc1 = DecisionTreeClassifier(random_state=12345, max_depth=3)
model_dtc1.fit(train_features, train_target)

DecisionTreeClassifier(max_depth=3, random_state=12345)

In [9]:
# код ревьюера, смотрим глубину дерева, которое обучалось без ограничений

model_dtc.get_depth()

24

Попробуем при лучшей глубине дерева = 3 поменять гиперпараметр `splitter` на `random`

In [10]:
#создаем объект
model_dtc2 = DecisionTreeClassifier(random_state=12345, max_depth=3, splitter='random')

#обучаем модель
model_dtc2.fit(train_features, train_target)

#предсказание
valid_predictions_dtc2 = model_dtc2.predict(valid_features)

#считаем метрику 
valid_accuracy_dtc2 = accuracy_score(valid_target, valid_predictions_dtc2)
valid_accuracy_dtc2

0.7387247278382582

Значение `accuracy` подуменьшилось. Оставим гиперпараметр `splitter` как есть. Попробуем поменять гиперпараметр `criterion` с `gini` на `entropy`

In [11]:
#создаем объект
model_dtc3 = DecisionTreeClassifier(random_state=12345, max_depth=3, criterion='entropy')

#обучаем модель
model_dtc3.fit(train_features, train_target)

#предсказание
valid_predictions_dtc3 = model_dtc3.predict(valid_features)

#считаем метрику
valid_accuracy_dtc3 = accuracy_score(valid_target, valid_predictions_dtc3)
valid_accuracy_dtc3

0.7853810264385692

Что меняй критерий, что не меняй: изменений никаких. Остановимся на модели с гиперпараметром `criterion` = `gini` и гиперпараметром `splitter` = `best`,  и глубиной дерева = 3.

**Случайный лес**

Попробуем алгоритм "случайный лес"

In [12]:
#создаем объект
model_rfc = RandomForestClassifier(random_state=12345)

#обучаем модель
model_rfc.fit(train_features, train_target)

#предсказание
valid_predictions_rfc = model_rfc.predict(valid_features)

#считаем метрику
valid_accuracy_rfc = accuracy_score(valid_target, valid_predictions_rfc)
valid_accuracy_rfc

0.7853810264385692

Значение такое же, как и у последнего варианта "дерева решений". Попробуем улучшить. Найдем оптимальное кол-во деревьев, например, из 200, которое дает лучшее значение `accuracy`.

In [13]:
best_max_est = 0
best_max_est_accuracy = 0

for est in range(1,201):
    model_rfc1 = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_rfc1.fit(train_features, train_target)
    valid_predictions_rfc1 = model_rfc1.predict(valid_features)
    valid_accuracy_rfc1 = accuracy_score(valid_target, valid_predictions_rfc1)
    if valid_accuracy_rfc1 > best_max_est_accuracy:
        best_max_est = est
        best_max_est_accuracy = valid_accuracy_rfc1

print('Лучшая accuracy', best_max_est_accuracy, 'при количестве', best_max_est)

Лучшая accuracy 0.7947122861586314 при количестве 23


In [14]:
model_rfc1 = RandomForestClassifier(random_state=12345, n_estimators=23)
model_rfc1.fit(train_features, train_target)

RandomForestClassifier(n_estimators=23, random_state=12345)

Попробуем поменять гиперпараметр `criterion` с `gini` на `entropy`

In [15]:
#создаем объект
model_rfc2 = RandomForestClassifier(random_state=12345, n_estimators=23, criterion='entropy')

#обучаем модель
model_rfc2.fit(train_features, train_target)

#предсказание
valid_predictions_rfc2 = model_rfc2.predict(valid_features)

#считаем метрику
valid_accuracy_rfc2 = accuracy_score(valid_target, valid_predictions_rfc2)
valid_accuracy_rfc2

0.7838258164852255

Остановимся на модели с гиперпараметром `criterion` = `gini` и количеством деревьев = 23

**Логистическая регрессия**

Попробуем логистическую регрессию

In [16]:
#создаем объект
model_lr = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)

#обучаем модель
model_lr.fit(train_features, train_target)

#предсказание
valid_predictions_lr = model_lr.predict(valid_features)

valid_accuracy_lr = accuracy_score(valid_target, valid_predictions_lr)
valid_accuracy_lr

0.7107309486780715

Accuracy хуже, чем у двух предыдущих алгоритмов. Перебирать гиперпараметры не будем. 

Самое высокое качество - 0.795 - у модели `RandomForestClassifier` c гиперпараметром `n_estimators` = 23: вместо одного решающего дерева используются целых 23. На втором месте модель `DecisionTreeClassifier`- 0.785 - дерево решений с гиперпараметром `max_depth` = 3. Самое низкое качество - 0.71 - у модели `LogisticRegression`. Поэтому, учитывая все вышеописанное, лучшей моделью при исследовании признается `RandomForestClassifier` c гиперпараметром `n_estimators` = 23. Ее-то и посмотрим на тестовой выборке.

## Проверьте модель на тестовой выборке

In [17]:
#выделим признаки и целевой признак из тестовой выборки
test_features = df_test.drop('is_ultra', 1)
test_target = df_test['is_ultra']

Протестируем лучшую модель "дерева решений"

In [18]:
test_predictions_dtc = model_dtc1.predict(test_features)

test_accuracy_dtc = accuracy_score(test_target, test_predictions_dtc)
test_accuracy_dtc

0.7791601866251944

Протестируем лучшую модель "случайного леса"

In [19]:
test_predictions_rfc = model_rfc1.predict(test_features)

test_accuracy_rfc = accuracy_score(test_target, test_predictions_rfc)
test_accuracy_rfc

0.7807153965785381

Протестируем логистическую регрессию

In [20]:
test_predictions_lr = model_lr.predict(test_features)

test_accuracy_lr = accuracy_score(test_target, test_predictions_lr)
test_accuracy_lr

0.6842923794712286

Самое высокое качество - 0.78 - у случайного леса: вместо одного решающего дерева используется целых 23.
На втором месте - 0.73 - дерево решений с глубиной дерева = 3. Самое низкое качество - 0.68 - у логистической регрессии. А вот по скорости работы все наоборот: в связи с минимальным набором параметров на первом месте  - логистическая регрессия, на втором  - дерево решений с глубиной=3 и на последнем - случайный лес с 23мя деревьями. На самом деле обработка 23 деревьев не так много времени занимает, поэтому лучшей моделью для данной задачи считается `RandomForestClassifier`.

## (бонус) Проверьте модели на адекватность

Проверим `RandomForestClassifier` на адекватность с помощью модели `DummyClassifier` на тестовых данных

In [21]:
model_dc = DummyClassifier(random_state=12345)

model_dc.fit(test_features, test_target)

test_predictions_dc = model_dc.predict(test_features)

test_accuracy_dc = accuracy_score(test_target, test_predictions_dc)
test_accuracy_dc

0.6842923794712286

У модели `DummyClassifier` на тестовых данных accuracy меньше, чем у  модели `RandomForestClassifier` на тех же тестовых данных, поэтому считаем, что модель `RandomForestClassifier` адекватна

## Итоговый вывод

Данные были предоставленны чистыми. Предобработка не потребовалась. Были исследованы три модели: `DecisionTreeClassifier`, `RandomForestClassifier` и `LinearRegression`. Наилучшую эффективность показала модель `RandomForestClassifier` с гиперпараметром `n_estimators` = 23. `RandomForestClassifier` была проверена на тестовой выборке и показала результат accuracy - 0.7807153965785381. Также `RandomForestClassifier`была проверена на адекватность с помощью сравнения с моделью `DummyClassifier` на тестовой выборке, которая показала результат accuracy = 0.6842923794712286.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откройте-и-изучите-файл" data-toc-modified-id="Откройте-и-изучите-файл-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откройте и изучите файл</a></span></li><li><span><a href="#Разбейте-данные-на-выборки" data-toc-modified-id="Разбейте-данные-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбейте данные на выборки</a></span></li><li><span><a href="#Исследуйте-модели" data-toc-modified-id="Исследуйте-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследуйте модели</a></span></li><li><span><a href="#Проверьте-модель-на-тестовой-выборке" data-toc-modified-id="Проверьте-модель-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверьте модель на тестовой выборке</a></span></li><li><span><a href="#(бонус)-Проверьте-модели-на-адекватность" data-toc-modified-id="(бонус)-Проверьте-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>(бонус) Проверьте модели на адекватность</a></span></li><li><span><a href="#Итоговый-вывод" data-toc-modified-id="Итоговый-вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Итоговый вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>